In [1]:
import pandas as pd

import sys
sys.path.append('/Users/basileroth/Desktop/Code/offseason_greenlobby/')
from offseason_greenlobby.config import color_16
import numpy as np
from offseason_greenlobby.utils.utils import get_surrounding_words,extract_until_period,clean_text,get_gp_senat
# define subplot grid
import matplotlib.pyplot as plt 
import re


In [2]:
LIST_COLS = ["IDEES","DESCRIPTION","NOM_AMENDEMENT","EXPOSE_SOMMAIRE",'article_courte',
       'article', 'type_amendement', 'groupe_politique_ref',
       'groupe_politique', 'groupe_politique_nom', 'type_auteur', 'acteur_ref',
       'acteur_nom', 'acteur_prenom', 'acteur_denom']

# CONSTANTES 

In [3]:
ARTICLE = 2

In [4]:
IDEE = "limiter les recours abusifs"

In [5]:
DESCRIPTION = "L'idée est de limiter au maximum les recours en réduisant les délais d'une part et en les rendant risqués car si il est considéré comme abusif, le requérant devra verser des dommages et intérêts.Plus précisément, afin de limiter les recours abusifs, lorsque le recours contre une autorisation environnementale est mis en œuvre dans des conditions qui traduisent un comportement abusif de la part du requérant et causent un préjudice au bénéficiaire de l’autorisation, ce dernier peut demander au juge administratif des dommages et intérêts. Par ailleurs, afin de limiter le risque de contentieux, le délai de recours n’est pas prorogé par l’exercice d’un recours administratif."

# Assemblée nationale

In [6]:
df_an = pd.read_csv("/Users/basileroth/Desktop/Code/offseason_greenlobby/data/processed/LOI_VERTE/AN/df_an.csv")
df_an.dropna(subset=["expose_sommaire"],inplace=True)
df_an["IDEES"] = IDEE
df_an["DESCRIPTION"] = DESCRIPTION
df_an["CHAMBRE"] = "AN"

df_an.rename(columns={
    "acteur_denom":"auteur",
    "expose_sommaire":"EXPOSE_SOMMAIRE",
    "nom_du_fichier":"NOM_AMENDEMENT"
},inplace=True)

df_an.rename(columns={
    "groupe_politique":"groupe_politique_abv",
},inplace=True)

df_an.rename(columns={
    "groupe_politique_nom":"groupe_politique",
},inplace=True)

df_an = df_an[df_an.article == ARTICLE]

df_an.shape

(180, 17)

### Sénat

In [8]:
df_senat_total = pd.read_csv("/Users/basileroth/Desktop/Code/offseason_greenlobby/data/processed/LOI_VERTE/SENAT/jeu_complet_2022-2023_737.csv", 
   encoding='ISO-8859-1',
   skiprows=1,
   sep='\t')

print("Senat : ",df_senat_total.shape[0])

df_senat_cm = pd.read_csv("/Users/basileroth/Desktop/Code/offseason_greenlobby/data/processed/LOI_VERTE/SENAT/jeu_complet_2023-2024_19.csv", 
   encoding='ISO-8859-1',
   skiprows=1,
   sep='\t')

print("Senat comissions: ",df_senat_cm.shape[0])

df_senat_cmp = pd.read_csv("/Users/basileroth/Desktop/Code/offseason_greenlobby/data/processed/LOI_VERTE/SENAT/jeu_complet_commission_2022-2023_607.csv", 
   encoding='ISO-8859-1',
   skiprows=1,
   sep='\t')

print("CMP comissions: ",df_senat_cmp.shape[0])

df_senat_total = pd.concat([df_senat_total,df_senat_cm,df_senat_cmp],axis=0)

df_senat_total.dropna(subset=['Objet '],inplace=True)

df_senat_total['Dispositif_c'] = df_senat_total['Dispositif '].apply(lambda x : clean_text(x))
df_senat_total['Objet_c'] = df_senat_total['Objet '].apply(lambda x : clean_text(x))


df_senat_total['article'] = df_senat_total['Subdivision '].apply(lambda x : ''.join(re.findall(r'\d+', x)))
df_senat_total.loc[df_senat_total['article'] == '',"article"] = 0
df_senat_total['article'] = df_senat_total['article'].apply(lambda x : int(x))

df_senat_total["IDEES"] = IDEE
df_senat_total["DESCRIPTION"] = DESCRIPTION

df_senat_total.rename(columns={
    "Auteur ":"auteur",
    "Objet_c":"EXPOSE_SOMMAIRE",
    "Numéro ":"NOM_AMENDEMENT"
},inplace=True)

df_senat_total["CHAMBRE"] = "SENAT"
# df_senat_total = df_senat_total[df_senat_total.article == ARTICLE]
df_senat_total['groupe_politique'] = df_senat_total['Fiche Sénateur'].astype(str).apply(lambda x : get_gp_senat( "https://"+x[2:]) if len(x) > 5 else "Inconnu")

df_senat_total.shape

Senat :  410
Senat comissions:  2
CMP comissions:  378
https://www.senat.fr/senfic/salmon_daniel20092g.html
Membre du groupe
Écologiste - Solidarité et Territoires
https://www.senat.fr/senfic/canayer_agnes14053l.html
Rattachée au
groupe Les Républicains
https://www.senat.fr/senfic/demilly_stephane17229f.html
Membre du groupe
Union Centriste
https://www.senat.fr/senfic/salmon_daniel20092g.html
Membre du groupe
Écologiste - Solidarité et Territoires
https://www.senat.fr/senfic/salmon_daniel20092g.html
Membre du groupe
Écologiste - Solidarité et Territoires
https://www.senat.fr/senfic/salmon_daniel20092g.html
Membre du groupe
Écologiste - Solidarité et Territoires
https://www.senat.fr/senfic/breuiller_daniel20914v.html
Membre du Groupe Écologiste - Solidarité et
Territoires
https://www.senat.fr/senfic/breuiller_daniel20914v.html
Membre du Groupe Écologiste - Solidarité et
Territoires
https://www.senat.fr/senfic/breuiller_daniel20914v.html
Membre du Groupe Écologiste - Solidarité et
Territ

(732, 20)

In [ ]:
# df_senat_total.to_csv("/Users/basileroth/Desktop/Code/offseason_greenlobby/data/processed/LOI_VERTE/SENAT/df_senat.csv")

# Concat SENAT + AN

In [8]:
pd.concat([
    df_senat_total[["IDEES","DESCRIPTION","NOM_AMENDEMENT","EXPOSE_SOMMAIRE","auteur","groupe_politique","CHAMBRE"]],
    df_an[["IDEES","DESCRIPTION","NOM_AMENDEMENT","EXPOSE_SOMMAIRE","auteur","groupe_politique","CHAMBRE"]],
]).to_csv("../../data/processed/LOI_VERTE/df_art"+str(ARTICLE)+".csv")